# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [34]:
# parameters:
kneibors36 = ''            # run the pipeline with kneibors and n_neighbors=36
kneibors_gs = ''           # run the knn with gridsearch
randomforest_gs = 'X'       # run the randomforest with gridsearch
stack_gs = ''             # stack more than one model

In [35]:
# import libraries
import pandas as pd
import nltk
from sklearn.pipeline import Pipeline, FeatureUnion
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import re
import numpy as np
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /Users/d5mit/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/d5mit/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [36]:
# load data from database
database_filepath = 'sqlite:////Users/d5mit/PycharmProjects/Udacity_ETL/Project/workspace/data/DisasterResponse.db'

df = pd.read_sql_table('MessageClass', database_filepath)  

X = df.iloc[:, 1:2].values[:,0]
y = df.iloc[:, 4:].values

In [37]:
X

array(['Weather update - a cold front from Cuba that could pass over Haiti',
       'Is the Hurricane over or is it not over',
       'Looking for someone but no name', ...,
       "Proshika, operating in Cox's Bazar municipality and 5 other unions, Ramu and Chokoria, assessment, 5 kg rice, 1,5 kg lentils to 700 families.",
       'Some 2,000 women protesting against the conduct of the elections were teargassed as they tried to converge on the local electoral commission offices in the southern oil city of Port Harcourt.',
       'A radical shift in thinking came about as a result of this meeting, recognizing that HIV/AIDS is at the core of the humanitarian crisis and identifying the crisis itself as a function of the HIV/AIDS pandemic.'],
      dtype=object)

In [38]:
y.shape

(26216, 36)

In [39]:
y

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

### 2. Write a tokenization function to process your text data

In [40]:
def replace_text_regex(text, iregex, iplaceholder):
    """ 
    replace texts based on regular expresion 

    Parameters: 
    text (string): Text that will be used to replace
    iregex (string): The regular expresion 
    iplaceholder (string): If the regular expresion is found, the text is replaced with the placeholder texts

    Returns: 
    the modified text
    """     
    
    # remove URLs and replace with "urlplaceholder"
    url_regex = iregex
    
    # get list of all urls using regex
    detected_urls = re.findall(url_regex, text)
    
    # replace each url in text string with placeholder
    for url in detected_urls:
        text = text.replace(url, iplaceholder)
    return text    

def tokenize(text):
    """ 
    Takes text and clean it
    - Remove urls and replace with "urlplaceholder"
    - Remove twitter tages and replace with "twitterplaceholder"
    - Replace the String based on the pattern -> replace number with string
    - Lemmatize text
    - Return clean_tokens

    Parameters: 
    clean_tokens (list): text as a list

    Returns: 
    the modified text
    """  
    
    # remove urls and replace with "urlplaceholder"
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'  
    text = replace_text_regex(text, url_regex, 'urlplaceholder')

    # remove urls and replace with "urlplaceholder"
    url_regex = 'www.?(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'  
    text = replace_text_regex(text, url_regex, 'urlplaceholder')
    
    # remove twitter tages and replace with "twitterplaceholder"
    url_regex = '//t.co?/(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'    
    text = replace_text_regex(text, url_regex, 'twitterplaceholder')
    
    # Replace the String based on the pattern -> replace number with string
    text = re.sub('[-+]?[0-9]+' , 'inumber', text).upper()
    
    aaa = 'aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa'
    # Replace the String based on the pattern
    text = text.replace(aaa, '')    
    
    # Replace the String based on the pattern
    text = text.replace('+', '')       
    
    # Replace the String based on the pattern
    text = text.replace('.', '')     
 
    # Replace the String based on the pattern
    text = text.replace("'", '') 

    # Replace the String based on the pattern
    text = text.replace("!", '') 

    # Replace the String based on the pattern
    text = text.replace("#", '') 

    # Replace the String based on the pattern
    text = text.replace("(", '') 
    
    # Replace the String based on the pattern
    text = text.replace(")", '')     
 
    # Replace the String based on the pattern
    text = text.replace("*", '')  

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    stop_words = stopwords.words('english') 

    
    clean_tokens = []
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]    
 
    punctuations="#?:.,;!$%&"
#     print(tokens)
    
    for tok in tokens:
        if tok in punctuations:
            tokens.remove(tok)
            
    for tok in tokens:
        if tok in punctuations:
            tokens.remove(tok)            
#     print(tokens)    
    
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
    
        clean_tokens.append(clean_tok)

    return clean_tokens

tokenize('This is :one/sentence?!#.')

['this', 'is', 'one/sentence']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [41]:
# split test and train data
print('(   )')
X_train, X_test, y_train, y_test = train_test_split(X, y)    

print(type(X_train))

cv = CountVectorizer(tokenizer=tokenize)

cv.fit_transform(X_train)

print(cv.get_feature_names())
print(len(cv.get_feature_names()))

(   )
<class 'numpy.ndarray'>
["''", '-', '--', '-_-', '-__-', '-a', '-an', '-baby', '-blankets', '-bottle', '-bottles', '-bouncy', '-breast-feeding', '-bumbo', '-candles', '-canned', '-cutaneous', '-droughts', '-fever', '-food', '-fresno', '-half', '-huffington', '-hygiene', '-magnitude', '-new', '-northjerseycom', '-operator-tractor-exp', '-other', '-paper', '-perishable', '-rice', '-saint-nicolas', '-size', '-st', '-the', '-tools', '-toys', '-we', '-west', '-wipes', '/', '//', '////', '//crap', '//estaba', '/federation', '/or', '/via', '/water', '<', '=', '=ainumber', '>', '?', '@', '[', '\\', '\\delva', '\\nhappy', '\\nin', '\\nthanks', '\\nthe', '\\nyes', ']', '_', '__', '___', '____', '_____', '_________', '___bol', '___rt', '___this', '___w/', '__amdatlady', '__basketcase', '__my', '__sandy', '__‚àö√ü', '_am_chile', '_before', '_gas', '_holy', '_jerseycity', '_korpiklaani', '_matt_inumber', '_maxwell_', '_mr_vargas', '_nobodyrealer', '_nyc', '_oh', '_rubeyg', '_sandy', '_staatz'

In [42]:
# pipeline.get_params()

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [43]:
def display_results(df, y_test, y_pred, idetails=False):
    """ 
    Takes the predicted value and compare it to the test value. (y_pred vs y_test)
    - for each column use the classification_report function to calculate the 'macro avg': f1 score, recall and precision
    - Get the average of all the  f1 score, recall and precisions

    Parameters: 
    df (dataframe): dataframe containing the columns
    y_test (numpy.ndarray): the test values (Test the predicted values against these values)
    y_pred (numpy.ndarray):  the predicted values
    idetails=False (boolean): output details

    Returns: 
    prints the: f1 score, recall and precision
    """      
    
    icolumns = df.columns[4:40].values
    counter = 0                 # counts the number of F1 scores
    total_f1 = 0                # calculates the sum of all F1 scores
    average_f1 = 0              # average F1 for over all columns
    total_precision = 0
    total_recall = 0
    average_precision = 0
    average_recall = 0
    
    for column in icolumns:
        
        # get F1 scores
        report = classification_report(y_test[counter], y_pred[counter],output_dict=True)
        
        print(report)
        
        # use macro see blog:
        # https://towardsdatascience.com/accuracy-precision-recall-or-f1-331fb37c5cb9
        # Use F1
        macro_precision =  report['macro avg']['precision'] 
        macro_recall = report['macro avg']['recall']    
        macro_f1 = report['macro avg']['f1-score']
        
        # print output
        if idetails == True:
            print(column)            
            print('F1 Score:', macro_f1)
            print('')            
        total_f1 = total_f1 + macro_f1
        total_precision = total_precision + macro_precision
        total_recall = total_recall + macro_recall
        counter = counter + 1

    average_f1 = total_f1 / counter
    print('f1 score: ', average_f1)
    
    average_recall = total_recall / counter
    print('Recall score: ', average_recall)
    
    average_precision = total_precision / counter
    print('Precision score: ', average_precision)    


### KNeibors 36:

In [44]:
# kneibors with 36 neibors

def build_model_KNN36():
    # build pipeline
    pipeline = Pipeline([
                         ('vect', CountVectorizer(tokenizer=tokenize)),
                         ('tfidf', TfidfTransformer()), 
                         ('clf', MultiOutputClassifier(KNeighborsClassifier(n_neighbors=5, leaf_size=20)))]) 
    print(pipeline.get_params())
    return pipeline

# Best Parameters: {'clf__estimator__leaf_size': 20, 'clf__estimator__n_neighbors': 5}

In [45]:
# kneibors with 36 neibors
if kneibors36 == 'X':
    # train classifier
    model = build_model_KNN36() 
    print('(#  )')
    model.fit(X_train, y_train)
    



In [46]:
# kneibors with 36 neibors
if kneibors36 == 'X':
    
    print('(## )')
    # predict on test data
    y_pred = model.predict(X_test)    
    print('(###)')        
    
    y_test
    


In [47]:
# kneibors with 36 neibors
if kneibors36 == 'X':
    display_results(df, y_test, y_pred, idetails=True)
    
df.columns[4:40].values

array(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers',
       'other_infrastructure', 'weather_related', 'floods', 'storm',
       'fire', 'earthquake', 'cold', 'other_weather', 'direct_report'],
      dtype=object)

In [48]:
y_test

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 1, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [49]:
# y_pred

### 6. Improve your model
Use grid search to find better parameters. 

### KNN:

In [ ]:
def build_model_KNN():
    # build pipeline
    pipeline = Pipeline([
                     ('vect', CountVectorizer(tokenizer=tokenize)),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultiOutputClassifier(KNeighborsClassifier()))])

    parameters = {
            'clf__estimator__n_neighbors': [5, 10, 15, 20],
            'clf__estimator__leaf_size': [20, 30, 40]
    }

    cv = GridSearchCV(pipeline, param_grid=parameters, verbose=10)
    print(cv.get_params())
    return cv

if kneibors_gs == 'X': 
    print('(   ) - Build')
    model = build_model_KNN()
    print('(#  ) - Train')
    model.fit(X_train, y_train)

    model.get_params()

(   ) - Build
{'cv': None, 'error_score': nan, 'estimator__memory': None, 'estimator__steps': [('vect', CountVectorizer(tokenizer=<function tokenize at 0x7fb571b683a0>)), ('tfidf', TfidfTransformer()), ('clf', MultiOutputClassifier(estimator=KNeighborsClassifier()))], 'estimator__verbose': False, 'estimator__vect': CountVectorizer(tokenizer=<function tokenize at 0x7fb571b683a0>), 'estimator__tfidf': TfidfTransformer(), 'estimator__clf': MultiOutputClassifier(estimator=KNeighborsClassifier()), 'estimator__vect__analyzer': 'word', 'estimator__vect__binary': False, 'estimator__vect__decode_error': 'strict', 'estimator__vect__dtype': <class 'numpy.int64'>, 'estimator__vect__encoding': 'utf-8', 'estimator__vect__input': 'content', 'estimator__vect__lowercase': True, 'estimator__vect__max_df': 1.0, 'estimator__vect__max_features': None, 'estimator__vect__min_df': 1, 'estimator__vect__ngram_range': (1, 1), 'estimator__vect__preprocessor': None, 'estimator__vect__stop_words': None, 'estimator_

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  clf__estimator__leaf_size=20, clf__estimator__n_neighbors=5, score=0.260, total= 1.5min
[CV] clf__estimator__leaf_size=20, clf__estimator__n_neighbors=5 .....


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.5min remaining:    0.0s


[CV]  clf__estimator__leaf_size=20, clf__estimator__n_neighbors=5, score=0.249, total= 1.5min
[CV] clf__estimator__leaf_size=20, clf__estimator__n_neighbors=5 .....


...

[CV]  clf__estimator__leaf_size=20, clf__estimator__n_neighbors=10, score=0.219, total= 1.4min
[CV] clf__estimator__leaf_size=20, clf__estimator__n_neighbors=10 ....


[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed: 11.6min remaining:    0.0s


[CV]  clf__estimator__leaf_size=20, clf__estimator__n_neighbors=10, score=0.236, total= 1.4min
[CV] clf__estimator__leaf_size=20, clf__estimator__n_neighbors=10 ....


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed: 13.0min remaining:    0.0s


[CV]  clf__estimator__leaf_size=20, clf__estimator__n_neighbors=10, score=0.225, total= 1.9min
[CV] clf__estimator__leaf_size=20, clf__estimator__n_neighbors=15 ....
[CV]  clf__estimator__leaf_size=20, clf__estimator__n_neighbors=15, score=0.227, total= 1.9min
[CV] clf__estimator__leaf_size=20, clf__estimator__n_neighbors=15 ....
[CV]  clf__estimator__leaf_size=20, clf__estimator__n_neighbors=15, score=0.219, total= 1.4min
[CV] clf__estimator__leaf_size=20, clf__estimator__n_neighbors=15 ....
[CV]  clf__estimator__leaf_size=20, clf__estimator__n_neighbors=15, score=0.202, total= 1.4min
[CV] clf__estimator__leaf_size=20, clf__estimator__n_neighbors=15 ....
[CV]  clf__estimator__leaf_size=20, clf__estimator__n_neighbors=15, score=0.226, total= 1.4min
[CV] clf__estimator__leaf_size=20, clf__estimator__n_neighbors=15 ....
[CV]  clf__estimator__leaf_size=20, clf__estimator__n_neighbors=15, score=0.215, total= 1.4min
[CV] clf__estimator__leaf_size=20, clf__estimator__n_neighbors=20 ....
[CV]

In [ ]:
if kneibors_gs == 'X':
    print('(## ) - Predict')
    y_pred = model.predict(X_test)

    print('(###) - Done')
    display_results(df, y_test, y_pred)
    print("\nBest Parameters:", model.best_params_)
    
    display_results(df, y_test, y_pred, idetails=True)
    


## Random Forest:

In [ ]:


# Number of trees in random forest
n_estimators = [10, 50, 100]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [10, 50, 100]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 3]
# Minimum number of samples required at each leaf node
min_samples_leaf = [2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]



def build_model_random_forest():
    pipeline = Pipeline([
                     ('vect', CountVectorizer(tokenizer=tokenize)),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])
    
#     # Create the random grid
#     random_grid = {'n_estimators': n_estimators,
#                    'max_features': max_features,
#                    'max_depth': max_depth,
#                    'min_samples_split': min_samples_split,
#                    'min_samples_leaf': min_samples_leaf,
#                    'bootstrap': bootstrap}
    parameters = {
                         'clf__estimator__n_estimators': n_estimators
#                          'clf__estimator__max_features': max_features
#                          'clf__estimator__max_depth': max_depth,
#                          'clf__estimator__min_samples_split': min_samples_split,
#                          'clf__estimator__min_samples_leaf': min_samples_leaf,
#                          'clf__estimator__bootstrap': bootstrap
    }
    
    
#     cv = RandomizedSearchCV(pipeline, param_distributions=parameters, verbose=10)    
    
    cv = GridSearchCV(pipeline, param_grid=parameters, verbose=10)
    print(cv.get_params().keys())    
    return cv



In [ ]:
cv.get_params()

In [ ]:
if randomforest_gs:
    print('Fit...')
    model_rf = build_model_random_forest()
    model_rf.get_params()
    model_rf.fit(X_train, y_train)    

In [ ]:
if randomforest_gs:
    print('Predict...')
    y_pred = model_rf.predict(X_test)

    print("Best parameters set found on development set:")
    print(model_rf.best_params_)
    display_results(df, y_test, y_pred, idetails=True)    

## Voting method:

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.naive_bayes import MultinomialNB


# Number of trees in random forest
n_estimators = [10, 50, 100]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [10, 50, 100]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 3]
# Minimum number of samples required at each leaf node
min_samples_leaf = [2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]



def build_model_stack():
    
    # Create Base Learners
    classifiers = [
                     ('rf_1', RandomForestClassifier(n_estimators=10, random_state=42)),
                     ('rf_2', KNeighborsClassifier(n_neighbors=5)),
                     ('rf_3', MultinomialNB()),
                    ]

    clf = VotingClassifier(classifiers, n_jobs=-1, voting='soft')

# ensemble=VotingClassifier(estimators=[('Decision Tree', decisiontree), ('Random Forest', forest)], 
#                        voting='soft', weights=[2,1]).fit(train_X,train_Y)    
    
    pipeline = Pipeline([
                     ('vect', CountVectorizer(tokenizer=tokenize)),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultiOutputClassifier(clf))
    ])
    
#     # Create the random grid
#     random_grid = {'n_estimators': n_estimators,
#                    'max_features': max_features,
#                    'max_depth': max_depth,
#                    'min_samples_split': min_samples_split,
#                    'min_samples_leaf': min_samples_leaf,
#                    'bootstrap': bootstrap}
    parameters = {
#                           'estimator__clf__estimator__final_estimator__n_estimators': [10]
#                          'clf__estimator__max_features': max_features
#                          'clf__estimator__max_depth': max_depth,
#                          'clf__estimator__min_samples_split': min_samples_split,
#                          'clf__estimator__min_samples_leaf': min_samples_leaf,
#                          'clf__estimator__bootstrap': bootstrap
    }
    
    
#     cv = RandomizedSearchCV(pipeline, param_distributions=parameters, verbose=10)    
    
    cv = GridSearchCV(pipeline, param_grid=parameters, verbose=10)
    print(cv.get_params().keys())    
    return pipeline


In [ ]:
if stack_gs == 'X':
    print('Fit...')
    model_st = build_model_stack()
    model_st.get_params()
    model_st.fit(X_train, y_train)  




In [ ]:
if stack_gs == 'X':
    print('Predict...')
    y_pred = model_st.predict(X_test)

    print("Best parameters set found on development set:")
#     print(model_st.best_params_)
    display_results(df, y_test, y_pred, idetails=True)    

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!